In [0]:
%pip install -r ../requirements.txt

In [0]:
dbutils.widgets.text("new_schema", defaultValue="finance")
dbutils.widgets.text("region", defaultValue="eu-west-1")
dbutils.widgets.text("catalog", defaultValue="production")
dbutils.widgets.text("uc_service_credential", defaultValue="production-aws-secrets-manager")
dbutils.widgets.text("secrets_manager_arn", defaultValue="")

metastore_id = sql("SELECT element_at(split(current_metastore(), ':'), -1) AS metastore").first().metastore
catalog = dbutils.widgets.get("catalog")
region = dbutils.widgets.get("region")
schema = dbutils.widgets.get("new_schema")
service_credential = dbutils.widgets.get("uc_service_credential")
secrets_manager_arn = dbutils.widgets.get("secrets_manager_arn")

In [0]:
import boto3

crypto_functions = dbutils.import_notebook("notebooks.envelope_encryption_v2.common.crypto_functions")

boto3_session = boto3.Session(botocore_session=dbutils.credentials.getServiceCredentialsProvider(service_credential), region_name=dbutils.widgets.get("region"))
secrets = crypto_functions.get_aws_secret(boto3_session, secrets_manager_arn)

In [0]:
dek = crypto_functions.generate_dek()
encrypted_dek = crypto_functions.encrypt_with_kek(
    kek_password=secrets.get("kek_password"), 
    kek_salt=secrets.get("kek_salt"), 
    to_encrypt=dek.get("private_key"))
encrypted_dek["dek"] = encrypted_dek.get("encrypted_string")
del dek["private_key"]
del encrypted_dek["encrypted_string"]
updated_secret = {f"{schema}_{k}": v for k, v in (encrypted_dek | dek).items()}

In [0]:
import json

response = crypto_functions.put_aws_secret(boto3_session, secrets_manager_arn, json.dumps(secrets|updated_secret))
response